In [3]:

from pyspark.sql import SparkSession
import os
import cml.data_v1 as cmldata


db_name= os.environ["DBNAME"]
tbl_name = []

#db_path =   "s3a://vr-uat1/warehouse/tablespace/external/hive"
#path_prefix = "s3a://vr-uat1/data/bank/"

db_path = os.environ["DBPATH"]
path_prefix = os.environ["PATHPREFIX"]
CONNECTION_NAME = os.environ["CONNECTION_NAME"]
tbl_name = []

#db_name= "vr_" + "testdb"
#db_path =   "s3a://vr-uat1/warehouse/tablespace/external/hive"

# spark = (SparkSession
#     .builder
#     .appName("bank-demo")
#     .config("spark.sql.warehouse.dir", db_path)
#     .config("spark.hadoop.fs.s2a.s3guard.ddb.region", "us-east-1")
#     .config('spark.sql.legacy.timeParserPolicy', 'LEGACY')
#     .config("spark.yarn.access.hadoopFileSystems","s3a://vr-uat1/")
#     .master("local[5]") # should be possible to change this to SPARK on Yarn or SPARK on Kubernetes
#     .getOrCreate())

conn = cmldata.get_connection(CONNECTION_NAME)
spark = conn.get_spark_session()


In [4]:
spark.catalog.listDatabases()

[Database(name='airlines', description='', locationUri='s3a://goes-se-sandbox01/warehouse/tablespace/external/hive/airlines.db'),
 Database(name='airlines_csv', description='', locationUri='s3a://goes-se-sandbox01/warehouse/tablespace/external/hive/airlines_csv.db'),
 Database(name='dbt_hive_demo', description='', locationUri='s3a://goes-se-sandbox01/warehouse/tablespace/external/hive/dbt_hive_demo.db'),
 Database(name='dbt_hive_demo1', description='', locationUri='s3a://goes-se-sandbox01/warehouse/tablespace/external/hive/dbt_hive_demo1.db'),
 Database(name='dbt_hive_demo1_mart_covid', description='', locationUri='s3a://goes-se-sandbox01/warehouse/tablespace/external/hive/dbt_hive_demo1_mart_covid.db'),
 Database(name='dbt_hive_demo1_reference', description='', locationUri='s3a://goes-se-sandbox01/warehouse/tablespace/external/hive/dbt_hive_demo1_reference.db'),
 Database(name='dbt_hive_demo1_staging_covid', description='', locationUri='s3a://goes-se-sandbox01/warehouse/tablespace/ext

In [5]:
spark.sql(f"SHOW TABLES FROM {db_name}").show()

+---------+-----------------+-----------+
|namespace|        tableName|isTemporary|
+---------+-----------------+-----------+
|vr_testdb|            trans|      false|
|vr_testdb|             loan|      false|
|vr_testdb|          account|      false|
|vr_testdb|             card|      false|
|vr_testdb|           client|      false|
|vr_testdb|           orders|      false|
|vr_testdb|      disposition|      false|
|vr_testdb|         district|      false|
|vr_testdb|    trans_cleaned|      false|
|vr_testdb|loan_full_cleaned|      false|
|vr_testdb|    order_cleaned|      false|
|vr_testdb|   client_cleaned|      false|
+---------+-----------------+-----------+



In [6]:
spark.sql(f"select count(*) from {db_name}.trans").show()

+--------+
|count(1)|
+--------+
| 1056320|
+--------+



## Objective : We want to segment the customer base to identify the High value customers

** References - ["Research Paper"](https://www.researchgate.net/profile/Herna-Viktor/publication/228949135_Mining_relational_databases_with_multi-view_learning/links/0fcfd506d9766253e1000000/Mining-relational-databases-with-multi-view-learning.pdf?origin=publication_detail )


In [7]:
spark.sql(f"select status, count(*) from {db_name}.loan group by status").show(30)

+------+--------+
|status|count(1)|
+------+--------+
|     B|      31|
|     D|      45|
|     C|     403|
|     A|     203|
+------+--------+



### Data model 
![](./images/DataModel.png?raw=true "Data  Model")


### Data Description
**Entity-Relationship Description **
- Each account has both static characteristics (e.g. date of creation, address of the branch) given in relation "account" and dynamic characteristics (e.g. payments debited or credited, balances) given in relations "permanent order" and "transaction".
- Relation "client" describes characteristics of persons who can manipulate with the accounts.
- One client can have more accounts, more clients can manipulate with single account; clients and accounts are related together in relation "disposition".
- Relations "loan" and "credit card" describe some services which the bank offers to its clients;
- More than one credit card can be issued to an account,
- At most one loan can be granted for an account.
- Relation "demographic data" gives some publicly available information about the districts (e.g. the unemployment rate); additional information about the clients can be deduced from this

[credits: webpages.charlotte.edu](https://webpages.charlotte.edu/mirsad/itcs6265/group1/domain.html)  


## ( Machine ) Learning Problem Description: 
### Classification of the loans 
Total : 682 Records, with status reflecting status of the loans
- A: Finished but good
- B: Finished but Bad
- C: good but not finished- 
- D: Bad and not finished



Background of every loan can be identified using Account, Client, Order,Transaction, Card, Disposition and District. 

Learning Problems we will work on here : 
1. If a loan is good or bad from 234 Finished laons
2. Use this to classify risk on the other existing Loans the loan is good or bad from 682 instances 


### Predict likelihood of accepting Credit Cards ( e.g. if we run a marketing campaign how likely is the user likely to accept credit cards)



In [8]:
### Exploratory Data Analysis and Feature Engineering

spark.sql(f"SELECT * FROM {db_name}.disposition").show(20)

+-------+---------+----------+---------+
|disp_id|client_id|account_id|     type|
+-------+---------+----------+---------+
|      1|        1|         1|    OWNER|
|      2|        2|         2|    OWNER|
|      3|        3|         2|DISPONENT|
|      4|        4|         3|    OWNER|
|      5|        5|         3|DISPONENT|
|      6|        6|         4|    OWNER|
|      7|        7|         5|    OWNER|
|      8|        8|         6|    OWNER|
|      9|        9|         7|    OWNER|
|     10|       10|         8|    OWNER|
|     11|       11|         8|DISPONENT|
|     12|       12|         9|    OWNER|
|     13|       13|        10|    OWNER|
|     14|       14|        11|    OWNER|
|     15|       15|        12|    OWNER|
|     16|       16|        12|DISPONENT|
|     17|       17|        13|    OWNER|
|     18|       18|        13|DISPONENT|
|     19|       19|        14|    OWNER|
|     20|       20|        15|    OWNER|
+-------+---------+----------+---------+
only showing top

### Data Preprocessing and Quality Checks.

** Business Rules usually set up by a Business Analyst/SME 
- Ensure Every Account has an Owner  ( i.e. a record in Account table also has one in Owner)
- Ensure Order / Loan exists in Transactions ( i.e. Records found in Order and Loan tables are also existing Transactions table)


In [9]:
# Write simple SQL query to check that 

# Checking that there are no orphaned account IDs (without Owners)
sql_string = f"""
    SELECT A.*, B.account_id  FROM {db_name}.account A
        LEFT OUTER JOIN {db_name}.disposition B ON A.account_id = B.account_id WHERE B.account_id IS NULL """

spark.sql(sql_string).show()

## You can remove IS NULL above to see the difference

+----------+-----------+---------+----+----------+
|account_id|district_id|frequency|date|account_id|
+----------+-----------+---------+----+----------+
+----------+-----------+---------+----+----------+



In [10]:
# check that Every record in Order and Loan table also exists as a transaction
spark.sql(f"""
    
           SELECT loan_ds.loan_id, loan_ds.account_id, loan_ds.date, loan_ds.amount, loan_ds.duration, loan_ds.payments, loan_ds.status, trans_ds.account_id,ROUND(trans_ds.total,0 ) as total FROM {db_name}.loan AS loan_ds
            LEFT OUTER JOIN 
          ( SELECT  DISTINCT account_id, sum(amount) as total  FROM  {db_name}.trans WHERE K_symbol = 'UVER'  GROUP BY account_Id , balance) AS  trans_ds
         ON loan_ds.account_id = trans_ds.account_id
         WHERE trans_ds.account_id IS NULL     
          """).show()

# Uncomment the line WHERE trans_ds.account_id ISS 

+-------+----------+----+------+--------+--------+------+----------+-----+
|loan_id|account_id|date|amount|duration|payments|status|account_id|total|
+-------+----------+----+------+--------+--------+------+----------+-----+
+-------+----------+----+------+--------+--------+------+----------+-----+



In [11]:
# Do some basic null checks on key tables
df_trans_acct = spark.sql(
    f""" 
        SELECT "trans" as Tablename , "account_Id", count(account_id) AS Total FROM {db_name}.trans WHERE account_id IS NULL
    """)
df_trans_date = spark.sql(
    f""" 
        SELECT "trans" as Tablename , "date", count(date) AS Total FROM {db_name}.trans WHERE date IS NULL
    """)

df_trans_acct.union(df_trans_date).show()

+---------+----------+-----+
|Tablename|account_Id|Total|
+---------+----------+-----+
|    trans|account_Id|    0|
|    trans|      date|    0|
+---------+----------+-----+



In [12]:
spark.sql(f"select * from {db_name}.card").show()

+-------+-------+-------+---------------+
|card_id|disp_id|   type|         issued|
+-------+-------+-------+---------------+
|   1005|   9285|classic|931107 00:00:00|
|    104|    588|classic|940119 00:00:00|
|    747|   4915|classic|940205 00:00:00|
|     70|    439|classic|940208 00:00:00|
|    577|   3687|classic|940215 00:00:00|
|    377|   2429|classic|940303 00:00:00|
|    721|   4680| junior|940405 00:00:00|
|    437|   2762|classic|940601 00:00:00|
|    188|   1146|classic|940619 00:00:00|
|     13|     87|classic|940629 00:00:00|
|    732|   4763|classic|940721 00:00:00|
|    181|   1066|classic|940819 00:00:00|
|    384|   2475| junior|940915 00:00:00|
|    309|   1946|classic|940919 00:00:00|
|    478|   3084|classic|941009 00:00:00|
|    563|   3589|classic|941021 00:00:00|
|    369|   2363|classic|941108 00:00:00|
|    376|   2428|classic|941110 00:00:00|
|    483|   3115|classic|941124 00:00:00|
|    174|   1039| junior|941204 00:00:00|
+-------+-------+-------+---------

### Preprocessing Step 1 : 
Examples of pre-processing issues that we now have to handle on the data
- Client Information : Extract age, gender 
- Transaction Table : Convert Transaction Information (type, operation , ksymbol)
- Convert Date Types

### Preprocessing Step 2: 
We will create 2 Data sets ( Loan and Credit card ) to then do some data analysis on these 2 types of businesses 

In [13]:
#Preprocessing Step 1: 
client_df = spark.sql(f"SELECT * from {db_name}.client")
client_df.show()

+---------+------+------------+-----------+
|client_id|gender|birth_number|district_id|
+---------+------+------------+-----------+
|        1|Female|      706213|         18|
|        2|  Male|      450204|          1|
|        3|Female|      406009|          1|
|        4|  Male|      561201|          5|
|        5|  Male|      605703|          5|
|        6|  Male|      190922|         12|
|        7|Female|      290125|         15|
|        8|Female|      385221|         51|
|        9|Female|      351016|         60|
|       10|  Male|      430501|         57|
|       11|Female|      505822|         57|
|       12|  Male|      810220|         40|
|       13|  Male|      745529|         54|
|       14|Female|      425622|         76|
|       15|Female|      185828|         21|
|       16|  Male|      190225|         21|
|       17|  Male|      341013|         76|
|       18|  Male|      315405|         76|
|       19|  Male|      421228|         47|
|       20|Female|      790104| 

In [14]:
#Birth number is an encoded variable containing Birthday and Sex: 
# Value : YYMMDD ( for Men)
# Value YYMM+50DD ( for Women)

# let us apply Spark Transform function here 
from pyspark.sql.functions import *
from pyspark.sql.types import StringType, BooleanType, DateType, IntegerType


# client_df.withColumn("newb", when(col("gender") == "Female" & col("birth_number").substr(3,2) > 50 , col("birth_number") - 5000).otherwise(col("birth_number"))).show()
client_df = client_df.withColumn("birth_numberU", 
#          when((col("gender") == "Female") & (col("birth_number").substr(3,2) > 50), col("birth_number") - 5000)
          when((col("birth_number").substr(3,2) > 50), col("birth_number") - 5000)
         .otherwise( col("birth_number")).cast(IntegerType()))

client_df = client_df.withColumn("birth_date", to_date(concat(lit("19"), col("birth_numberU").cast(StringType())), "yyyyMMdd"))
client_df = client_df.drop(col("birth_numberU"))
client_df.show()


+---------+------+------------+-----------+----------+
|client_id|gender|birth_number|district_id|birth_date|
+---------+------+------------+-----------+----------+
|        1|Female|      706213|         18|1970-12-13|
|        2|  Male|      450204|          1|1945-02-04|
|        3|Female|      406009|          1|1940-10-09|
|        4|  Male|      561201|          5|1956-12-01|
|        5|  Male|      605703|          5|1960-07-03|
|        6|  Male|      190922|         12|1919-09-22|
|        7|Female|      290125|         15|1929-01-25|
|        8|Female|      385221|         51|1938-02-21|
|        9|Female|      351016|         60|1935-10-16|
|       10|  Male|      430501|         57|1943-05-01|
|       11|Female|      505822|         57|1950-08-22|
|       12|  Male|      810220|         40|1981-02-20|
|       13|  Male|      745529|         54|1974-05-29|
|       14|Female|      425622|         76|1942-06-22|
|       15|Female|      185828|         21|1918-08-28|
|       16

In [15]:
# We see that we have some issues with data
df2 = client_df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull(), c \
                           )).alias(c)
                    for c in client_df.columns])

df2.show()

+---------+------+------------+-----------+----------+
|client_id|gender|birth_number|district_id|birth_date|
+---------+------+------------+-----------+----------+
|        0|     0|           0|          0|         0|
+---------+------+------------+-----------+----------+



In [16]:
# let us now merge the client, disp and loans data
disp_df = spark.sql(f"SELECT * from {db_name}.disposition")
loan_df = spark.sql(f"SELECT * FROM {db_name}.loan")

disp_df.show(10)
loan_df.show(10)

loan_full_df = (client_df.join(disp_df, client_df.client_id == disp_df.client_id , "inner")
                         .join(loan_df, disp_df.disp_id == loan_df.loan_id, "inner")).drop(disp_df.client_id).drop(disp_df.account_id)
loan_full_df.show()

+-------+---------+----------+---------+
|disp_id|client_id|account_id|     type|
+-------+---------+----------+---------+
|      1|        1|         1|    OWNER|
|      2|        2|         2|    OWNER|
|      3|        3|         2|DISPONENT|
|      4|        4|         3|    OWNER|
|      5|        5|         3|DISPONENT|
|      6|        6|         4|    OWNER|
|      7|        7|         5|    OWNER|
|      8|        8|         6|    OWNER|
|      9|        9|         7|    OWNER|
|     10|       10|         8|    OWNER|
+-------+---------+----------+---------+
only showing top 10 rows

+-------+----------+------+------+--------+--------+------+
|loan_id|account_id|  date|amount|duration|payments|status|
+-------+----------+------+------+--------+--------+------+
|   5314|      1787|930705| 96396|      12| 8033.00|     B|
|   5316|      1801|930711|165960|      36| 4610.00|     A|
|   6863|      9188|930728|127080|      60| 2118.00|     A|
|   5325|      1843|930803|105804|      

In [17]:

spark.sql(f"SELECT * FROM {db_name}.trans WHERE operation IS NOT NULL").show()



+--------+----------+------+------+-------------+-------+-------+--------+----+--------+
|trans_id|account_id|  date|  type|    operation| amount|balance|k_symbol|bank| account|
+--------+----------+------+------+-------------+-------+-------+--------+----+--------+
|  695247|      2378|930101|PRIJEM|        VKLAD| 700.00| 700.00|    null|null|    null|
|  171812|       576|930101|PRIJEM|        VKLAD| 900.00| 900.00|    null|null|    null|
|  207264|       704|930101|PRIJEM|        VKLAD|1000.00|1000.00|    null|null|    null|
| 1117247|      3818|930101|PRIJEM|        VKLAD| 600.00| 600.00|    null|null|    null|
|  579373|      1972|930102|PRIJEM|        VKLAD| 400.00| 400.00|    null|null|    null|
|  771035|      2632|930102|PRIJEM|        VKLAD|1100.00|1100.00|    null|null|    null|
|  452728|      1539|930103|PRIJEM|        VKLAD| 600.00| 600.00|    null|null|    null|
|  725751|      2484|930103|PRIJEM|        VKLAD|1100.00|1100.00|    null|null|    null|
|  497211|      1695|

### Transformation : We now need to transform the Type type, Operation, K_symbol fields to readable formats

In [18]:
import numpy as np
from pyspark.sql.types import StringType
 
@udf(returnType=StringType())    
def convert_trans_type_to_eng(x):
    if x == 'PRIJEM':
        return 'Credit'
    elif x == 'VYDAJ':
        return 'Withdrawal'
    else:
        return np.NaN
    
@udf(returnType=StringType())    
def convert_trans_op_to_eng(x):
    if x == 'VYBER KARTOU':
        return 'Credit card withdrawal'
    elif x == 'VKLAD':
        return 'Credit in cash'
    elif x == 'PREVOD Z UCTU':
        return 'Collection from another bank'
    elif x == 'VYBER':
        return 'Withdrawal in Cash'
    elif x == 'PREVOD NA UCET':
        return 'Remittance to another bank'    
    else:
        return np.NaN
@udf(returnType=StringType())        
def convert_trans_k_symbol_to_eng(x):
    if x == 'POJISTNE':
        return 'Insurance payment'
    elif x == 'SLUZBY':
        return 'Payment for statement'
    elif x == 'UROK':
        return 'Interest credited'
    elif x == 'SANKC. UROK':
        return 'Sanction interest if negative balance'
    elif x == 'SIPO':
        return 'Household'
    elif x == 'DUCHOD':
        return 'Old-age pension'  
    elif x == 'UVER':
        return 'Loan payment'      
    else:
        return np.NaN

In [19]:

trans_df = spark.sql(f"SELECT * FROM {db_name}.trans WHERE operation IS NOT NULL")

trans_df.show()

trans_df = trans_df.withColumn("trans_date",  to_date(concat(lit("19"), col("date").cast(StringType())), "yyyyMMdd")) \
        .select("trans_id", "account_id", "date", "type", "operation", "amount", "balance", "k_symbol", "bank", "account", "trans_date", \
                        convert_trans_type_to_eng(col("type")).alias("type_eng" ), \
                        convert_trans_op_to_eng(col("operation")).alias("operation_eng"), \
                        convert_trans_k_symbol_to_eng(col("k_symbol")).alias("k_symbol_eng"))
trans_df.show()



+--------+----------+------+------+-------------+-------+-------+--------+----+--------+
|trans_id|account_id|  date|  type|    operation| amount|balance|k_symbol|bank| account|
+--------+----------+------+------+-------------+-------+-------+--------+----+--------+
|  695247|      2378|930101|PRIJEM|        VKLAD| 700.00| 700.00|    null|null|    null|
|  171812|       576|930101|PRIJEM|        VKLAD| 900.00| 900.00|    null|null|    null|
|  207264|       704|930101|PRIJEM|        VKLAD|1000.00|1000.00|    null|null|    null|
| 1117247|      3818|930101|PRIJEM|        VKLAD| 600.00| 600.00|    null|null|    null|
|  579373|      1972|930102|PRIJEM|        VKLAD| 400.00| 400.00|    null|null|    null|
|  771035|      2632|930102|PRIJEM|        VKLAD|1100.00|1100.00|    null|null|    null|
|  452728|      1539|930103|PRIJEM|        VKLAD| 600.00| 600.00|    null|null|    null|
|  725751|      2484|930103|PRIJEM|        VKLAD|1100.00|1100.00|    null|null|    null|
|  497211|      1695|

+--------+----------+------+------+-------------+-------+-------+--------+----+--------+----------+--------+--------------------+---------------+
|trans_id|account_id|  date|  type|    operation| amount|balance|k_symbol|bank| account|trans_date|type_eng|       operation_eng|   k_symbol_eng|
+--------+----------+------+------+-------------+-------+-------+--------+----+--------+----------+--------+--------------------+---------------+
|  695247|      2378|930101|PRIJEM|        VKLAD| 700.00| 700.00|    null|null|    null|1993-01-01|  Credit|      Credit in cash|            NaN|
|  171812|       576|930101|PRIJEM|        VKLAD| 900.00| 900.00|    null|null|    null|1993-01-01|  Credit|      Credit in cash|            NaN|
|  207264|       704|930101|PRIJEM|        VKLAD|1000.00|1000.00|    null|null|    null|1993-01-01|  Credit|      Credit in cash|            NaN|
| 1117247|      3818|930101|PRIJEM|        VKLAD| 600.00| 600.00|    null|null|    null|1993-01-01|  Credit|      Credit in 

In [20]:
#Need to Clean Order table as well 
order_df = spark.sql(f"Select * from {db_name}.orders")
@udf(returnType=StringType())
def convert_order_k_symbol_to_eng(x):
    if x == 'POJISTNE':
        return 'Insurance payment'
    elif x == 'SIPO':
        return 'Household'
    elif x == 'LEASING':
        return 'Leasing'
    elif x == 'UVER':
        return 'Loan payment'
    else:
        return np.NaN
    
order_df = order_df.withColumn("k_symbol_eng",  convert_order_k_symbol_to_eng(col("k_symbol")))


In [21]:
# spark.sql("REFRESH TABLE vr_testdb.client").show()
spark.sql(f"SELECT * from {db_name}.client").show()

+---------+------+------------+-----------+
|client_id|gender|birth_number|district_id|
+---------+------+------------+-----------+
|        1|Female|      706213|         18|
|        2|  Male|      450204|          1|
|        3|Female|      406009|          1|
|        4|  Male|      561201|          5|
|        5|  Male|      605703|          5|
|        6|  Male|      190922|         12|
|        7|Female|      290125|         15|
|        8|Female|      385221|         51|
|        9|Female|      351016|         60|
|       10|  Male|      430501|         57|
|       11|Female|      505822|         57|
|       12|  Male|      810220|         40|
|       13|  Male|      745529|         54|
|       14|Female|      425622|         76|
|       15|Female|      185828|         21|
|       16|  Male|      190225|         21|
|       17|  Male|      341013|         76|
|       18|  Male|      315405|         76|
|       19|  Male|      421228|         47|
|       20|Female|      790104| 

In [22]:
#finally We need to write all this data as a cleaned dataset
(client_df.write 
          .mode("overwrite")
#          .option("path", db_path )
          .saveAsTable(db_name + "." + "client_cleaned", format="parquet"))
#spark.sql("REFRESH TABLE " + db_name + "." + "client_cleaned")
spark.sql("SELECT * from " + db_name + "." + "client_cleaned").show(3)

(trans_df.write 
           .mode("overwrite")
#           .option("path", db_path )
           .saveAsTable(db_name + "." + "trans_cleaned", format="parquet"))
# spark.sql("REFRESH TABLE " + db_name + "." + "trans_cleaned")
spark.sql("SELECT * from " + db_name + "." + "trans_cleaned").show(3)

(loan_full_df.write 
           .mode("overwrite")
#   .option("path", db_path )
           .saveAsTable(db_name + "." + "loan_full_cleaned", format="parquet"))
# #    spark.sql("REFRESH TABLE " + db_name + "." + table_name).show()
spark.sql("SELECT * from " + db_name + "." + "loan_full_cleaned").show(3)

(order_df.write 
           .mode("overwrite")
#   .option("path", db_path )
           .saveAsTable(db_name + "." + "order_cleaned", format="parquet"))
# #    spark.sql("REFRESH TABLE " + db_name + "." + table_name).show()
spark.sql("SELECT * from " + db_name + "." + "order_cleaned").show(3)


+---------+------+------------+-----------+----------+
|client_id|gender|birth_number|district_id|birth_date|
+---------+------+------------+-----------+----------+
|        1|Female|      706213|         18|1970-12-13|
|        2|  Male|      450204|          1|1945-02-04|
|        3|Female|      406009|          1|1940-10-09|
+---------+------+------------+-----------+----------+
only showing top 3 rows



+--------+----------+------+------+---------+-------+-------+--------+----+-------+----------+--------+--------------+------------+
|trans_id|account_id|  date|  type|operation| amount|balance|k_symbol|bank|account|trans_date|type_eng| operation_eng|k_symbol_eng|
+--------+----------+------+------+---------+-------+-------+--------+----+-------+----------+--------+--------------+------------+
|  695247|      2378|930101|PRIJEM|    VKLAD| 700.00| 700.00|    null|null|   null|1993-01-01|  Credit|Credit in cash|         NaN|
|  171812|       576|930101|PRIJEM|    VKLAD| 900.00| 900.00|    null|null|   null|1993-01-01|  Credit|Credit in cash|         NaN|
|  207264|       704|930101|PRIJEM|    VKLAD|1000.00|1000.00|    null|null|   null|1993-01-01|  Credit|Credit in cash|         NaN|
+--------+----------+------+------+---------+-------+-------+--------+----+-------+----------+--------+--------------+------------+
only showing top 3 rows



+---------+------+------------+-----------+----------+-------+-----+-------+----------+------+------+--------+--------+------+
|client_id|gender|birth_number|district_id|birth_date|disp_id| type|loan_id|account_id|  date|amount|duration|payments|status|
+---------+------+------------+-----------+----------+-------+-----+-------+----------+------+------+--------+--------+------+
|     4959|  Male|      790425|         77|1979-04-25|   4959|OWNER|   4959|         2|940105| 80952|      24| 3373.00|     A|
|     4961|  Male|      820731|         54|1982-07-31|   4961|OWNER|   4961|        19|960429| 30276|      12| 2523.00|     B|
|     4968|  Male|      380526|         50|1938-05-26|   4968|OWNER|   4968|        38|980419|110736|      48| 2307.00|     C|
+---------+------+------------+-----------+----------+-------+-----+-------+----------+------+------+--------+--------+------+
only showing top 3 rows



+--------+----------+-------+----------+-------+--------+------------+
|order_id|account_id|bank_to|account_to| amount|k_symbol|k_symbol_eng|
+--------+----------+-------+----------+-------+--------+------------+
|   29401|         1|     YZ|  87144583|2452.00|    SIPO|   Household|
|   29402|         2|     ST|  89597016|3372.70|    UVER|Loan payment|
|   29403|         2|     QR|  13943797|7266.00|    SIPO|   Household|
+--------+----------+-------+----------+-------+--------+------------+
only showing top 3 rows



In [23]:
spark.stop()

In [27]:
#check to see if the spark tables are persisted in the datalake. 
import cml.data_v1 as cmldata

CONNECTION_NAME = os.environ["CONNECTION_NAME"]
conn = cmldata.get_connection(CONNECTION_NAME)
spark = conn.get_spark_session()

spark.sql(f"select * from {db_name}.loan_full_cleaned").show()

Hive Session ID = 8281c004-7d58-4e30-94fd-f65e0434401c


+---------+------+------------+-----------+----------+-------+-----+-------+----------+------+------+--------+--------+------+
|client_id|gender|birth_number|district_id|birth_date|disp_id| type|loan_id|account_id|  date|amount|duration|payments|status|
+---------+------+------------+-----------+----------+-------+-----+-------+----------+------+------+--------+--------+------+
|     4959|  Male|      790425|         77|1979-04-25|   4959|OWNER|   4959|         2|940105| 80952|      24| 3373.00|     A|
|     4961|  Male|      820731|         54|1982-07-31|   4961|OWNER|   4961|        19|960429| 30276|      12| 2523.00|     B|
|     4968|  Male|      380526|         50|1938-05-26|   4968|OWNER|   4968|        38|980419|110736|      48| 2307.00|     C|
|     4989|  Male|      350904|         28|1935-09-04|   4989|OWNER|   4989|       105|981205|352704|      48| 7348.00|     C|
|     5002|Female|      775717|         65|1977-07-17|   5002|OWNER|   5002|       173|940531|104808|      12| 

In [26]:
spark.stop()